In [74]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import json
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

#### Fetch xml sitemap

In [75]:
sitemap = pd.read_xml('https://www.rtbf.be/site-map/articles.xml')
sitemap.head()

,loc,lastmod,changefreq,news,image
0,https://www.rtbf.be/article/direct-video-vietn...,2023-08-01T08:42:27.000Z,hourly,NaN,NaN
1,https://www.rtbf.be/article/couvin-un-nouveau-...,2023-08-01T08:36:40.000Z,hourly,NaN,NaN
2,https://www.rtbf.be/article/niger-la-france-va...,2023-08-01T08:29:48.000Z,hourly,NaN,NaN
3,https://www.rtbf.be/article/entre-douceur-et-d...,2023-08-01T08:27:08.000Z,hourly,NaN,NaN
4,https://www.rtbf.be/article/putsch-au-niger-l-...,2023-08-01T08:27:05.000Z,hourly,NaN,NaN


#### Keep only two first columns and change their name

In [76]:
df = sitemap.drop(['changefreq', 'news', 'image'],axis=1)
df.rename(columns={'loc':'source_url', 'lastmod':'last_modified_date'}, inplace=True)
df.head()

,source_url,last_modified_date
0,https://www.rtbf.be/article/direct-video-vietn...,2023-08-01T08:42:27.000Z
1,https://www.rtbf.be/article/couvin-un-nouveau-...,2023-08-01T08:36:40.000Z
2,https://www.rtbf.be/article/niger-la-france-va...,2023-08-01T08:29:48.000Z
3,https://www.rtbf.be/article/entre-douceur-et-d...,2023-08-01T08:27:08.000Z
4,https://www.rtbf.be/article/putsch-au-niger-l-...,2023-08-01T08:27:05.000Z


#### Keep only date in 'last_modified_date' column

In [77]:
df['last_modified_date'].replace({r"T.+": ''}, inplace=True, regex=True)
df.head()

,source_url,last_modified_date
0,https://www.rtbf.be/article/direct-video-vietn...,2023-08-01
1,https://www.rtbf.be/article/couvin-un-nouveau-...,2023-08-01
2,https://www.rtbf.be/article/niger-la-france-va...,2023-08-01
3,https://www.rtbf.be/article/entre-douceur-et-d...,2023-08-01
4,https://www.rtbf.be/article/putsch-au-niger-l-...,2023-08-01


#### Find published date function

In [97]:
def find_published_date(url: str) ->str:
    response = requests.get(url)
    soup = bs(response.content, 'html.parser')
    result= soup.find("script", {"type": "application/ld+json"})
    data = json.loads(result.text, strict=False)
    published_date = data['datePublished']
    return published_date

In [98]:
print(find_published_date(article_one))

2023-07-18T10:29:32+02:00


#### Find article title function

In [86]:
def find_article_title(url:str) -> str: 
    response = requests.get(url)
    soup = bs(response.content, 'html.parser')
    article_title = soup.find('h1').text
    return article_title

In [87]:
print(find_article_title(article_one))


Coupe du monde féminine de football 2023 : horaires, directs et résultats des 64 matches en Australie et Nouvelle-Zélande


#### Find article text function

In [89]:
def find_article_text(url: str) -> str:
    response = requests.get(url)
    soup = bs(response.content, 'html.parser')
    paragraphs = [p.text for p in soup.find_all('p', attrs={'class': None})]
    article_text = " ".join(paragraphs)
   
    return article_text    

In [90]:
print(find_article_text(article_one))

La neuvième édition de la Coupe du monde féminine bat son plein en Australie et en Nouvelle-Zélande depuis le 20 juillet dernier. Du spectacle, des jolis buts et des surprises, nous avons déjà été gâtés en ce début de Mondial 2023. Un spectacle à suivre en direct vidéo sur la RTBF. Avec une diffusion de toutes les rencontres ! Retrouvez toutes les rencontres, tous les résumés et tous les résultats ci-dessous. Tous les sujets de l'article


#### Add article_text column to df dataframe

In [91]:
df['article_text'] = df['source_url'].apply(find_article_text)
df.head()

,source_url,last_modified_date,article_text
0,https://www.rtbf.be/article/direct-video-vietn...,2023-08-01,La neuvième édition de la Coupe du monde fémin...
1,https://www.rtbf.be/article/couvin-un-nouveau-...,2023-08-01,Ce sont que des enfants livrés à eux-mêmes dan...
2,https://www.rtbf.be/article/niger-la-france-va...,2023-08-01,La France a décidé d’évacuer dès mardi ses res...
3,https://www.rtbf.be/article/entre-douceur-et-d...,2023-08-01,"Tout au long de l’été, nous vous proposons une..."
4,https://www.rtbf.be/article/putsch-au-niger-l-...,2023-08-01,"Au Niger, la situation est toujours extrêmemen..."


#### Add article_title to df dataframe

In [92]:
df['article_title'] = df['source_url'].apply(find_article_title)
df.head()

,source_url,last_modified_date,article_text,article_title
0,https://www.rtbf.be/article/direct-video-vietn...,2023-08-01,La neuvième édition de la Coupe du monde fémin...,Coupe du monde féminine de football 2023 : hor...
1,https://www.rtbf.be/article/couvin-un-nouveau-...,2023-08-01,Ce sont que des enfants livrés à eux-mêmes dan...,Couvin: un nouveau centre ouvre ses portes aux...
2,https://www.rtbf.be/article/niger-la-france-va...,2023-08-01,La France a décidé d’évacuer dès mardi ses res...,Niger : la France va débuter l’évacuation de s...
3,https://www.rtbf.be/article/entre-douceur-et-d...,2023-08-01,"Tout au long de l’été, nous vous proposons une...","Entre douceur et douleur, une playlist qui piq..."
4,https://www.rtbf.be/article/putsch-au-niger-l-...,2023-08-01,"Au Niger, la situation est toujours extrêmemen...","Putsch au Niger : ""L’échec de l’opération Bark..."


#### Add published_date column to df dataframe

In [99]:
df['published_date'] = df['source_url'].apply(find_published_date)

In [100]:
df.head()

,source_url,last_modified_date,article_text,article_title,published_date
0,https://www.rtbf.be/article/direct-video-vietn...,2023-08-01,La neuvième édition de la Coupe du monde fémin...,Coupe du monde féminine de football 2023 : hor...,2023-07-18T10:29:32+02:00
1,https://www.rtbf.be/article/couvin-un-nouveau-...,2023-08-01,Ce sont que des enfants livrés à eux-mêmes dan...,Couvin: un nouveau centre ouvre ses portes aux...,2023-08-01T10:36:40+02:00
2,https://www.rtbf.be/article/niger-la-france-va...,2023-08-01,La France a décidé d’évacuer dès mardi ses res...,Niger : la France va débuter l’évacuation de s...,2023-08-01T10:29:48+02:00
3,https://www.rtbf.be/article/entre-douceur-et-d...,2023-08-01,"Tout au long de l’été, nous vous proposons une...","Entre douceur et douleur, une playlist qui piq...",2023-08-01T10:27:08+02:00
4,https://www.rtbf.be/article/putsch-au-niger-l-...,2023-08-01,"Au Niger, la situation est toujours extrêmemen...","Putsch au Niger : ""L’échec de l’opération Bark...",2023-08-01T10:27:05+02:00


#### keep only date from published_date

In [101]:
df['published_date'].replace({r"T.+": ''}, inplace=True, regex=True)
df.head()

,source_url,last_modified_date,article_text,article_title,published_date
0,https://www.rtbf.be/article/direct-video-vietn...,2023-08-01,La neuvième édition de la Coupe du monde fémin...,Coupe du monde féminine de football 2023 : hor...,2023-07-18
1,https://www.rtbf.be/article/couvin-un-nouveau-...,2023-08-01,Ce sont que des enfants livrés à eux-mêmes dan...,Couvin: un nouveau centre ouvre ses portes aux...,2023-08-01
2,https://www.rtbf.be/article/niger-la-france-va...,2023-08-01,La France a décidé d’évacuer dès mardi ses res...,Niger : la France va débuter l’évacuation de s...,2023-08-01
3,https://www.rtbf.be/article/entre-douceur-et-d...,2023-08-01,"Tout au long de l’été, nous vous proposons une...","Entre douceur et douleur, une playlist qui piq...",2023-08-01
4,https://www.rtbf.be/article/putsch-au-niger-l-...,2023-08-01,"Au Niger, la situation est toujours extrêmemen...","Putsch au Niger : ""L’échec de l’opération Bark...",2023-08-01


#### Rearrange column order

In [104]:
final_df = df.loc[:, ['source_url', 'article_title', 'article_text', 'published_date', 'last_modified_date']]
final_df.head()

,source_url,article_title,article_text,published_date,last_modified_date
0,https://www.rtbf.be/article/direct-video-vietn...,Coupe du monde féminine de football 2023 : hor...,La neuvième édition de la Coupe du monde fémin...,2023-07-18,2023-08-01
1,https://www.rtbf.be/article/couvin-un-nouveau-...,Couvin: un nouveau centre ouvre ses portes aux...,Ce sont que des enfants livrés à eux-mêmes dan...,2023-08-01,2023-08-01
2,https://www.rtbf.be/article/niger-la-france-va...,Niger : la France va débuter l’évacuation de s...,La France a décidé d’évacuer dès mardi ses res...,2023-08-01,2023-08-01
3,https://www.rtbf.be/article/entre-douceur-et-d...,"Entre douceur et douleur, une playlist qui piq...","Tout au long de l’été, nous vous proposons une...",2023-08-01,2023-08-01
4,https://www.rtbf.be/article/putsch-au-niger-l-...,"Putsch au Niger : ""L’échec de l’opération Bark...","Au Niger, la situation est toujours extrêmemen...",2023-08-01,2023-08-01


#### Export to csv

In [106]:
final_df.to_csv('data/rtbf_articles.csv')